<a href="https://colab.research.google.com/github/Rostyslav-Surelo/Analytics/blob/main/ab_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
from statsmodels.stats.proportion import proportions_ztest
from google.colab import auth, drive
auth.authenticate_user()
client = bigquery.Client(project="data-analytics-mate")


In [ ]:
query = """
WITH
 session_info AS (
 SELECT
   s.date,
   s.ga_session_id,
   sp.country,
   sp.continent,
   sp.device,
   sp.channel,
   ab.test,
   ab.test_group
 FROM
   `DA.ab_test` ab
 JOIN
   `DA.session` s
 ON
   ab.ga_session_id = s.ga_session_id
 JOIN
   `DA.session_params` sp
 ON
   sp.ga_session_id = ab.ga_session_id ),


 session_with_orders AS (
 SELECT
   session_info.date,
   session_info.country,
   session_info.continent,
   session_info.device,
   session_info.channel,
   session_info.test,
   session_info.test_group,
   COUNT(DISTINCT o.ga_session_id) AS session_with_orders
 FROM
   `DA.order` o
 JOIN
   session_info
 ON
   o.ga_session_id = session_info.ga_session_id
 GROUP BY
   session_info.date,
   session_info.country,
   session_info.continent,
   session_info.device,
   session_info.channel,
   session_info.test,
   session_info.test_group
   ),


events as (
  select
   session_info.date,
   session_info.country,
   session_info.continent,
   session_info.device,
   session_info.channel,
   session_info.test,
   session_info.test_group,
   ep.event_name,
   COUNT(ep.ga_session_id) AS event_cnt
  from
  `DA.event_params` ep
 join
 session_info
 on ep.ga_session_id = session_info.ga_session_id
group by
   session_info.date,
   session_info.country,
   session_info.continent,
   session_info.device,
   session_info.channel,
   session_info.test,
   session_info.test_group,
   ep.event_name
),
session as (
select
   session_info.date,
   session_info.country,
   session_info.continent,
   session_info.device,
   session_info.channel,
   session_info.test,
   session_info.test_group,
   COUNT(DISTINCT session_info.ga_session_id) AS session_cnt
  from
  session_info
  group by
   session_info.date,
   session_info.country,
   session_info.continent,
   session_info.device,
   session_info.channel,
   session_info.test,
   session_info.test_group
),
account as (
select
   session_info.date,
   session_info.country,
   session_info.continent,
   session_info.device,
   session_info.channel,
   session_info.test,
   session_info.test_group,
   count(distinct acs.ga_session_id) as new_account_cnt
from
`DA.account_session` acs
join
session_info
on acs.ga_session_id = session_info.ga_session_id
group by
   session_info.date,
   session_info.country,
   session_info.continent,
   session_info.device,
   session_info.channel,
   session_info.test,
   session_info.test_group
)
select
   session_with_orders.date,
   session_with_orders.country,
   session_with_orders.continent,
   session_with_orders.device,
   session_with_orders.channel,
   session_with_orders.test,
   session_with_orders.test_group,
   'session with orders' as event_name,
   session_with_orders.session_with_orders as value
from
session_with_orders
union all
select
   events.date,
   events.country,
   events.continent,
   events.device,
   events.channel,
   events.test,
   events.test_group,
   event_name,
   events.event_cnt as value
from
events
union all
select
   session.date,
   session.country,
   session.continent,
   session.device,
   session.channel,
   session.test,
   session.test_group,
   'session' as event_name,
   session_cnt as value
from
session
union all
select
   account.date,
   account.country,
   account.continent,
   account.device,
   account.channel,
   account.test,
   account.test_group,
   'new account' as event_name,
   new_account_cnt as value
from
account;
"""
df = client.query(query).to_dataframe()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800996 entries, 0 to 800995
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        800996 non-null  dbdate
 1   country     800996 non-null  object
 2   continent   800996 non-null  object
 3   device      800996 non-null  object
 4   channel     800996 non-null  object
 5   test        800996 non-null  Int64 
 6   test_group  800996 non-null  Int64 
 7   event_name  800996 non-null  object
 8   value       800996 non-null  Int64 
dtypes: Int64(3), dbdate(1), object(5)
memory usage: 57.3+ MB


In [ ]:
def analysis_ab_test(data, metrics, denominator, segment_name="Total"):
    """
    Аналіз A/B тестів для кількох метрик.

    Параметри:
    - data: DataFrame з колонками ['test', 'test_group', 'event_name', 'value']
    - metrics: список метрик (event_name), які є чисельниками
    - denominator: одна подія (наприклад, 'session'), яка виступає знаменником
    - segment_name: назва сегменту для аналізу (наприклад, 'Total', 'Asia', 'Tablet')

    Повертає:
    - DataFrame з результатами по кожному тесту, метриці і групі
    """
    result = []

    for test_number in sorted(data['test'].unique()):
        for metric in metrics:
            # Чисельники
            num_g1 = data[(data['event_name'] == metric) & (data['test'] == test_number) & (data['test_group'] == 1)]['value'].sum()
            num_g2 = data[(data['event_name'] == metric) & (data['test'] == test_number) & (data['test_group'] == 2)]['value'].sum()

            # Знаменники
            denom_g1 = data[(data['event_name'] == denominator) & (data['test'] == test_number) & (data['test_group'] == 1)]['value'].sum()
            denom_g2 = data[(data['event_name'] == denominator) & (data['test'] == test_number) & (data['test_group'] == 2)]['value'].sum()

            # Розрахунок статистики
            if denom_g1 > 0 and denom_g2 > 0:
                count = np.array([num_g1, num_g2])
                nobs = np.array([denom_g1, denom_g2])
                z_stat, p_value = proportions_ztest(count, nobs)

                conv_g1 = num_g1 / denom_g1 * 100
                conv_g2 = num_g2 / denom_g2 * 100
                metric_change = (conv_g2 - conv_g1) / conv_g1 * 100 if conv_g1 > 0 else np.nan
                significant = p_value < 0.05
            else:
                z_stat = p_value = conv_g1 = conv_g2 = metric_change = np.nan
                significant = False

            result.append({
                'test_number': test_number,
                'segment': segment_name,
                'metric': f"{metric}/{denominator}",
                'numerator_event': metric,
                'denominator': denominator,
                'numerator_count_group1': num_g1,
                'denominator_count_group1': denom_g1,
                'conversion_rate_group1': conv_g1,
                'numerator_count_group2': num_g2,
                'denominator_count_group2': denom_g2,
                'conversion_rate_group2': conv_g2,
                'metric_change': metric_change,
                'z_stat': abs(z_stat),
                'p_value': p_value,
                'significant': significant
            })

    return pd.DataFrame(result)


In [ ]:
metrics = ['add_payment_info', 'add_shipping_info', 'begin_checkout', 'new account']
denominator = 'session'

segments = [
    (df, "Total"),
    (df[df['channel'] == 'Organic Search'], "Organic Search"),
    (df[df['channel'] == 'Direct'], "Direct"),
    (df[df['channel'] == 'Paid Search'], "Paid Search"),
    (df[df['device'] == 'desktop'], "Desktop"),
    (df[df['device'] == 'tablet'], "Tablet"),
    (df[df['continent'] == 'Asia'], "Asia"),
    (df[df['continent'] == 'Europe'], "Europe")
]

all_results = []

for seg_df, seg_name in segments:
    seg_result = analysis_ab_test(seg_df, metrics, denominator, segment_name=seg_name)
    all_results.append(seg_result)

df_all = pd.concat(all_results, ignore_index=True)


In [ ]:
drive.mount('/content/drive')
df_all.to_csv('/content/drive/MyDrive/ab_test.csv', index=False)


#Посилання на [Tableau](https://public.tableau.com/app/profile/rostyslav.surelo/viz/abtestresult/Significance?publish=yes)